基本的なJPEG変換と保存

In [2]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

def dicom_to_jpeg(dicom_path, output_path, quality=95):
    """DICOM画像をJPEG形式で保存"""
    
    # DICOMファイルを読み込み
    ds = pydicom.dcmread(dicom_path)
    img = ds.pixel_array
    
    # 画像の正規化（0-255の範囲に）
    img_normalized = normalize_dicom_image(img, ds)
    
    # PIL Imageに変換
    pil_image = Image.fromarray(img_normalized)
    
    # JPEG形式で保存
    pil_image.save(output_path, 'JPEG', quality=quality)
    
    print(f"JPEG画像を保存しました: {output_path}")
    return output_path

def normalize_dicom_image(img, ds):
    """DICOM画像を正規化"""
    
    # ウィンドウレベル調整
    window_center = ds.WindowCenter
    window_width = ds.WindowWidth
    
    min_val = window_center - window_width // 2
    max_val = window_center + window_width // 2
    
    # 画像をクリッピング
    img_clipped = np.clip(img, min_val, max_val)
    
    # 0-255の範囲に正規化
    img_normalized = ((img_clipped - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    
    return img_normalized

# 使用例
dicom_file = 'sample/sample.dcm'
output_file = 'sample/sample.jpg'

dicom_to_jpeg(dicom_file, output_file, quality=95)

JPEG画像を保存しました: sample/sample.jpg


'sample/sample.jpg'

2. 複数のDICOMファイルを一括変換

In [ ]:
import glob
import os

def batch_dicom_to_jpeg(input_folder='sample', output_folder='sample_jpeg', quality=95):
    """フォルダー内の全DICOMファイルをJPEGに変換"""
    
    # 出力フォルダーを作成
    os.makedirs(output_folder, exist_ok=True)
    
    # DICOMファイルを検索
    dicom_files = glob.glob(os.path.join(input_folder, '*.dcm'))
    
    if not dicom_files:
        print(f"'{input_folder}'フォルダーにDICOMファイルが見つかりません")
        return
    
    print(f"見つかったDICOMファイル数: {len(dicom_files)}")
    
    # 各ファイルを変換
    for dicom_file in dicom_files:
        try:
            # ファイル名を取得
            base_name = os.path.splitext(os.path.basename(dicom_file))[0]
            output_file = os.path.join(output_folder, f"{base_name}.jpg")
            
            # JPEGに変換
            dicom_to_jpeg(dicom_file, output_file, quality)
            
        except Exception as e:
            print(f"エラー ({dicom_file}): {e}")
    
    print(f"変換完了！出力フォルダー: {output_folder}")

# 一括変換の実行
batch_dicom_to_jpeg()

3. 高品質なJPEG変換（医療画像用）

In [3]:
def high_quality_dicom_to_jpeg(dicom_path, output_path, quality=100):
    """高品質なJPEG変換（医療画像用）"""
    
    ds = pydicom.dcmread(dicom_path)
    img = ds.pixel_array
    
    # より精密な正規化
    img_normalized = advanced_normalization(img, ds)
    
    # PIL Imageに変換
    pil_image = Image.fromarray(img_normalized, mode='L')  # グレースケール
    
    # 高品質JPEGで保存
    pil_image.save(output_path, 'JPEG', quality=quality, optimize=True)
    
    print(f"高品質JPEG画像を保存しました: {output_path}")

def advanced_normalization(img, ds):
    """高度な正規化処理"""
    
    # 画像の統計情報を取得
    img_min = img.min()
    img_max = img.max()
    
    # ヒストグラム平坦化（オプション）
    from skimage import exposure
    img_eq = exposure.equalize_hist(img)
    
    # 0-255に正規化
    img_normalized = (img_eq * 255).astype(np.uint8)
    
    return img_normalized

# 高品質変換の実行
high_quality_dicom_to_jpeg('sample/sample.dcm', 'sample/sample_hq.jpg')

高品質JPEG画像を保存しました: sample/sample_hq.jpg


C:\Users\pond1\AppData\Local\Temp\ipykernel_126560\4198958458.py:11: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil_image = Image.fromarray(img_normalized, mode='L')  # グレースケール


4. メタデータ付きJPEG保存

In [5]:
import json

def dicom_to_jpeg_with_separate_metadata(dicom_path, output_path, metadata_path):
    """JPEG画像とメタデータを別々に保存"""
    
    ds = pydicom.dcmread(dicom_path)
    img = ds.pixel_array
    
    # 画像を正規化
    img_normalized = normalize_dicom_image(img, ds)
    
    # PIL Imageに変換
    pil_image = Image.fromarray(img_normalized)
    
    # JPEGに保存
    pil_image.save(output_path, 'JPEG', quality=95)
    
    # メタデータをJSONファイルで保存
    metadata = {
        'PatientName': str(ds.PatientName),
        'StudyDate': str(ds.StudyDate),
        'Modality': str(ds.Modality),
        'ImageSize': f"{ds.Rows}x{ds.Columns}",
        'BitsAllocated': str(ds.BitsAllocated)
    }
    
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"JPEG画像を保存しました: {output_path}")
    print(f"メタデータを保存しました: {metadata_path}")

# 別々に保存の実行
dicom_to_jpeg_with_separate_metadata(
    'sample/sample.dcm', 
    'sample/sample.jpg', 
    'sample/sample_metadata.json'
)

JPEG画像を保存しました: sample/sample.jpg
メタデータを保存しました: sample/sample_metadata.json
